In [1]:
# @title Cell 1: LogDrainTrie Definition
import re
import math
from collections import defaultdict

class LogCluster:
    def __init__(self, cid, template_tokens):
        self.cid = cid
        self.template = template_tokens
        self.count = 0
        self.param_stats = defaultdict(lambda: {'sum': 0.0, 'sq_sum': 0.0, 'count': 0})

    def update_param(self, index, value):
        if not isinstance(value, (int, float)): return
        stats = self.param_stats[index]
        stats['sum'] += value
        stats['sq_sum'] += (value * value)
        stats['count'] += 1

class LogDrainTrie:
    """
    Log Parsing & Anomaly Detection.
    Threshold = 3.0 (Standard 3-Sigma Rule) -> Tối ưu hóa Recall.
    """
    def __init__(self, depth=4, sim_th=0.5, max_children=1000, rare_ratio=0.0, st_threshold=3.0):
        self.depth = depth
        self.sim_th = sim_th
        self.max_children = max_children
        self.rare_ratio = rare_ratio
        self.st_threshold = st_threshold # <--- CHỈNH VỀ 3.0 ĐỂ BẮT HẾT LỖI

        self.root = {}
        self.clusters = {}
        self.next_cid = 1
        self.total_logs = 0

        self.rex = [
            (r'blk_[-\d]+', '<BLK>'),
            (r'part-\d+', '<PART>'),
            (r'(\d{1,3}\.){3}\d{1,3}(:\d+)?', '<IP>'),
            (r'([0-9a-fA-F-]{36})', '<UUID>'),
            (r'user_\d+', '<USER>')
        ]

    def _tokenize(self, line):
        line = line.strip()
        match = re.search(r':\s+(.*)', line)
        content = match.group(1) if match else line
        for rex, tag in self.rex: content = re.sub(rex, tag, content)
        parts = re.split(r'[\s=:,\[\]\(\)]+', content)
        vals, tpls = [], []
        for t in parts:
            if not t: continue
            if t.replace('.','',1).isdigit() or (t.startswith('-') and t[1:].isdigit()):
                tpls.append('<NUM>'); vals.append(float(t))
            else:
                tpls.append(t); vals.append(t)
        return vals, tpls

    def _get_seq_sim(self, seq1, seq2):
        if not seq1 or not seq2: return 0
        matches = sum(1 for t1, t2 in zip(seq1, seq2) if t1==t2 or t1=='<*>' or t2=='<*>')
        return matches / max(len(seq1), 1)

    def _merge(self, seq1, seq2):
        return [t1 if t1 == t2 else '<*>' for t1, t2 in zip(seq1, seq2)]

    def _traverse(self, tokens, create_mode=False):
        length = len(tokens)
        if length not in self.root:
            if create_mode: self.root[length] = {}
            else: return None
        node = self.root[length]
        depth = 0
        for token in tokens:
            if depth >= self.depth: break
            if token in node: node = node[token]
            elif '<*>' in node: node = node['<*>']
            else:
                if create_mode:
                    if len(node) < self.max_children: node[token] = {}; node = node[token]
                    else:
                        if '<*>' not in node: node['<*>'] = {}
                        node = node['<*>']
                else: return None
            depth += 1
        return node

    def train(self, lines):
        self.total_logs = 0; self.root = {}; self.clusters = {}; self.next_cid = 1
        for line in lines:
            if not line.strip(): continue
            self.total_logs += 1
            vals, tokens = self._tokenize(line)
            if not tokens: continue
            node = self._traverse(tokens, create_mode=True)
            if 'cids' not in node: node['cids'] = []
            best_cid = -1; best_sim = -1.0
            for cid in node['cids']:
                cl = self.clusters[cid]
                sim = self._get_seq_sim(cl.template, tokens)
                if sim > best_sim and cl.template[0] == tokens[0]:
                    best_sim = sim; best_cid = cid
            if best_cid != -1 and best_sim >= self.sim_th:
                cluster = self.clusters[best_cid]
                cluster.count += 1
                cluster.template = self._merge(cluster.template, tokens)
                for i, (t_token, val) in enumerate(zip(tokens, vals)):
                    if t_token == '<NUM>': cluster.update_param(i, val)
            else:
                new_cl = LogCluster(self.next_cid, tokens)
                new_cl.count = 1
                for i, (t_token, val) in enumerate(zip(tokens, vals)):
                    if t_token == '<NUM>': new_cl.update_param(i, val)
                self.clusters[self.next_cid] = new_cl
                node['cids'].append(self.next_cid)
                self.next_cid += 1

    def detect(self, lines):
        results = []
        # Rare ratio: Nếu logs quá lớn, ngưỡng cứng 5 có thể quá nhỏ.
        # Dùng max(5, ...) để linh hoạt hơn.
        threshold_count = max(5, int(self.total_logs * self.rare_ratio))

        for line_idx, line in enumerate(lines):
            if not line.strip(): continue
            vals, tokens = self._tokenize(line)
            if not tokens: continue
            node = self._traverse(tokens, create_mode=False)
            target_cl = None; best_sim = -1.0
            if node and 'cids' in node:
                for cid in node['cids']:
                    cl = self.clusters[cid]
                    sim = self._get_seq_sim(cl.template, tokens)
                    if sim > best_sim and cl.template[0] == tokens[0]:
                        best_sim = sim; target_cl = cl

            is_anomaly = False

            # 1. Structural Check
            if not target_cl or best_sim < self.sim_th: is_anomaly = True
            elif target_cl.count <= threshold_count: is_anomaly = True

            # 2. Parameter Check (Z-Score > 3.0)
            else:
                for i, (t_token, val) in enumerate(zip(tokens, vals)):
                    if t_token == '<NUM>' and i in target_cl.param_stats:
                        stats = target_cl.param_stats[i]
                        # Chỉ check khi đã có mẫu thống kê
                        if stats['count'] > 2:
                            mean = stats['sum'] / stats['count']
                            variance = (stats['sq_sum'] / stats['count']) - (mean ** 2)
                            std_dev = math.sqrt(max(0, variance))

                            # Nếu std=0 (các giá trị trước đó giống hệt nhau)
                            # mà giá trị mới khác biệt -> Bắt luôn
                            if std_dev == 0:
                                if val != mean: is_anomaly = True; break
                            else:
                                z_score = abs(val - mean) / std_dev
                                if z_score > self.st_threshold:
                                    is_anomaly = True; break
            if is_anomaly: results.append(line_idx + 1)
        return results

In [2]:
# @title Cell 2: Main Execution Pipeline
import os
import time
from google.colab import files

def visualize_trie(node, depth=0, max_depth=3):
    """Hàm để in cấu trúc Trie, giúp em hình dung cách dữ liệu được nén."""
    if depth > max_depth: return
    indent = "  " * depth
    count = 0
    for token, child in node.items():
        if token in ['cids']: continue
        print(f"{indent}|- {token}")
        visualize_trie(child, depth + 1, max_depth)
        count += 1
        if count >= 3: # Chỉ in đại diện 3 nhánh để không bị tràn màn hình
            print(f"{indent}|- ... (hidden)")
            break

def save_and_download_report(lines, anomaly_indices, filename="anomaly_report.txt"):
    """Ghi kết quả ra file và kích hoạt download."""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(f"=== BAO CAO PHAT HIEN BAT THUONG ===\n")
            f.write(f"Tong so log: {len(lines)}\n")
            f.write(f"So luong bat thuong: {len(anomaly_indices)}\n")
            f.write(f"Ty le bat thuong: {(len(anomaly_indices)/len(lines))*100:.2f}%\n")
            f.write("-" * 50 + "\n\n")

            if not anomaly_indices:
                f.write("Khong phat hien dong bat thuong nao.\n")
            else:
                for idx in anomaly_indices:
                    # idx là số thứ tự dòng (bắt đầu từ 1), lines là mảng (bắt đầu từ 0)
                    line_content = lines[idx-1].strip()
                    f.write(f"[Line {idx}]: {line_content}\n")

        print(f"\n[System] Đã tạo file báo cáo: {filename}")
        files.download(filename)
    except Exception as e:
        print(f"Lỗi khi ghi file báo cáo: {e}")

def main():
    print("=== BƯỚC 1: NHẬP DỮ LIỆU LOG ===")
    print("Vui lòng upload file log (.txt, .log) của bạn...")
    uploaded = files.upload()

    if not uploaded:
        print("Chưa có file nào được upload.")
        return

    filename = list(uploaded.keys())[0]
    print(f"\n[System] Đang đọc file: {filename}...")

    try:
        # Dùng errors='replace' để tránh lỗi utf-8 với các log lạ
        with open(filename, 'r', encoding='utf-8', errors='replace') as f:
            lines = f.readlines()
    except Exception as e:
        print(f"Lỗi đọc file: {e}")
        return

    # Khởi tạo mô hình
    # st_threshold=3.0: Theo quy tắc 3-sigma (bắt các giá trị nằm ngoài 99.7% phân phối chuẩn)
    model = LogDrainTrie(depth=4, sim_th=0.5, rare_ratio=0.01, st_threshold=3.0)

    print(f"\n=== BƯỚC 2: TRAINING & COMPRESSION ===")
    start_time = time.time()
    model.train(lines)
    train_time = time.time() - start_time

    # Tính toán thống kê nén
    n_samples = model.total_logs
    n_templates = len(model.clusters)
    # Tỷ lệ nén = (1 - số mẫu / số dòng) * 100
    compression_rate = (1 - (n_templates / n_samples)) * 100 if n_samples > 0 else 0

    print(f"-> Thời gian Train: {train_time:.4f} giây")
    print(f"-> Tổng số dòng log: {n_samples}")
    print(f"-> Số lượng Cluster (Template) sinh ra: {n_templates}")
    print(f"-> Tỷ lệ nén (Compression Rate): {compression_rate:.2f}%")
    print("(Tỷ lệ nén cao chứng tỏ cấu trúc Trie đã gom nhóm hiệu quả)")

    print(f"\n=== BƯỚC 3: DETECTING ANOMALIES ===")
    detect_start = time.time()
    anomaly_indices = model.detect(lines)
    detect_time = time.time() - detect_start

    print(f"-> Thời gian Detect: {detect_time:.4f} giây")
    print(f"-> Phát hiện: {len(anomaly_indices)} dòng bất thường.")

    # In thử 5 dòng đầu tiên ra màn hình để check nhanh
    if anomaly_indices:
        print("\n--- Review nhanh 5 dòng bất thường đầu tiên ---")
        for idx in anomaly_indices[:5]:
            print(f"[Line {idx}]: {lines[idx-1].strip()[:100]}...")
    else:
        print("\n-> Không tìm thấy bất thường nào (Dữ liệu sạch hoặc ngưỡng quá cao).")

    print(f"\n=== BƯỚC 4: XUẤT BÁO CÁO & TRỰC QUAN HÓA ===")
    # 1. Trực quan hóa Trie (Sample)
    if model.root:
        # Lấy nhánh có độ dài phổ biến nhất để vẽ
        most_common_len = max(model.root.keys(), key=lambda k: len(model.root[k]))
        print(f"\n[Visual] Cấu trúc Trie (Sample cho độ dài {most_common_len}):")
        visualize_trie(model.root[most_common_len])

    # 2. Xuất file
    save_and_download_report(lines, anomaly_indices)

if __name__ == "__main__":
    main()

=== BƯỚC 1: NHẬP DỮ LIỆU LOG ===
Vui lòng upload file log (.txt, .log) của bạn...


Saving SSH.part001.txt to SSH.part001.txt

[System] Đang đọc file: SSH.part001.txt...

=== BƯỚC 2: TRAINING & COMPRESSION ===
-> Thời gian Train: 2.3142 giây
-> Tổng số dòng log: 100000
-> Số lượng Cluster (Template) sinh ra: 724
-> Tỷ lệ nén (Compression Rate): 99.28%
(Tỷ lệ nén cao chứng tỏ cấu trúc Trie đã gom nhóm hiệu quả)

=== BƯỚC 3: DETECTING ANOMALIES ===
-> Thời gian Detect: 2.2510 giây
-> Phát hiện: 4844 dòng bất thường.

--- Review nhanh 5 dòng bất thường đầu tiên ---
[Line 1]: ﻿Dec 10 06:55:46 LabSZ sshd[24200]: reverse mapping checking getaddrinfo for ns.marryaldkfaczcz.com ...
[Line 2]: Dec 10 06:55:46 LabSZ sshd[24200]: Invalid user webmaster from 173.234.31.186...
[Line 3]: Dec 10 06:55:46 LabSZ sshd[24200]: input_userauth_request: invalid user webmaster [preauth]...
[Line 9]: Dec 10 07:07:38 LabSZ sshd[24206]: Invalid user test9 from 52.80.34.196...
[Line 10]: Dec 10 07:07:38 LabSZ sshd[24206]: input_userauth_request: invalid user test9 [preauth]...

=== BƯỚC 4: XUẤT 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>